In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("customer_churn.csv")

In [3]:
df = df.drop('customerID',axis='columns')

In [4]:
df = df[df.TotalCharges!=' ']
df.TotalCharges = pd.to_numeric(df.TotalCharges)

In [5]:
df.replace('No phone service', 'No', inplace = True)
df.replace('No internet service', 'No', inplace = True)

In [6]:
df.replace('Yes', 1, inplace=True)
df.replace('No', 0, inplace=True)

In [7]:
df['gender'].replace({'Female':1,'Male':0}, inplace=True)

In [8]:
df_with_dummies=pd.get_dummies(data=df, columns=['InternetService', 'Contract','PaymentMethod'])

In [9]:
cols_to_scale = ['tenure','MonthlyCharges','TotalCharges']
scaler = MinMaxScaler()

df_with_dummies[cols_to_scale]=scaler.fit_transform(df_with_dummies[cols_to_scale])

In [10]:
df_X=df_with_dummies.drop('Churn', axis=1)
df_Y=df_with_dummies['Churn']

In [11]:
df_with_dummies.shape

(7032, 27)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.25, random_state=5)

In [13]:
def ANN(X_train, y_train, X_test, y_test, loss, weights) : 
    model=keras.Sequential([
        keras.layers.Dense(26, input_shape=(26,), activation='relu'),
        keras.layers.Dense(15, activation='relu'),
        
        keras.layers.Dense(1, activation='sigmoid'),
    ])
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy']) 
    
    if weights == -1 :
        model.fit(X_train, y_train, epochs = 10)
    else : 
        model.fit(X_train, y_train, epochs = 10, class_weight=weights)
        
    print(model.evaluate(X_test, y_test))
    
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred)
    
    print("Classification Report : ")
    print(classification_report(y_test, y_pred))
    
    return y_pred

In [14]:
y_pred = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/10
165/165 [==============================] - 2s 3ms/step - loss: 0.5424 - accuracy: 0.6964
Epoch 2/10
165/165 [==============================] - 1s 3ms/step - loss: 0.4261 - accuracy: 0.7969
Epoch 3/10
165/165 [==============================] - 1s 3ms/step - loss: 0.4168 - accuracy: 0.8036
Epoch 4/10
165/165 [==============================] - 1s 3ms/step - loss: 0.4135 - accuracy: 0.8060
Epoch 5/10
165/165 [==============================] - 1s 3ms/step - loss: 0.4094 - accuracy: 0.8091
Epoch 6/10
165/165 [==============================] - 1s 4ms/step - loss: 0.4085 - accuracy: 0.8091
Epoch 7/10
165/165 [==============================] - 1s 4ms/step - loss: 0.4063 - accuracy: 0.8113
Epoch 8/10
165/165 [==============================] - 1s 3ms/step - loss: 0.4045 - accuracy: 0.8119
Epoch 9/10
165/165 [==============================] - 1s 3ms/step - loss: 0.4024 - accuracy: 0.8127
Epoch 10/10
55/55 [==============================] - 0s 2ms/step - loss: 0.4400 - accuracy: 0.7833
[

In [15]:
# Method 1 : Under Sampling 

In [16]:
# Class count 
count_class_0, count_class_1 = df_with_dummies.Churn.value_counts()

In [17]:
# Divide by Class
df_class_0 = df_with_dummies[df_with_dummies['Churn']==0]
df_class_1 = df_with_dummies[df_with_dummies['Churn']==1]

In [18]:
df_class_0.shape

(5163, 27)

In [19]:
df_class_1.shape

(1869, 27)

In [20]:
count_class_0, count_class_1

(5163, 1869)

In [21]:
df_class_0.sample(count_class_1).shape

(1869, 27)

In [22]:
df_class_0_underSampled = df_class_0.sample(count_class_1)

In [23]:
df_underSampled = pd.concat([df_class_0_underSampled, df_class_1], axis=0)
df_underSampled.shape

(3738, 27)

In [24]:
print("Random Under Sampling : ")
print(df_underSampled.Churn.value_counts())

Random Under Sampling : 
0    1869
1    1869
Name: Churn, dtype: int64


In [25]:
x = df_underSampled.drop('Churn', axis=1)
y = df_underSampled['Churn']

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=13, stratify=y)

In [27]:
y_train.value_counts()

0    1495
1    1495
Name: Churn, dtype: int64

In [28]:
y_pred = ANN(x_train, y_train, x_test, y_test,'binary_crossentropy',-1)

Epoch 1/10
94/94 [==============================] - 1s 3ms/step - loss: 0.6166 - accuracy: 0.6819
Epoch 2/10
94/94 [==============================] - 0s 3ms/step - loss: 0.5134 - accuracy: 0.7512
Epoch 3/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4988 - accuracy: 0.7615
Epoch 4/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4923 - accuracy: 0.7659
Epoch 5/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4911 - accuracy: 0.7632
Epoch 6/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4854 - accuracy: 0.7669
Epoch 7/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4846 - accuracy: 0.7689
Epoch 8/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4829 - accuracy: 0.7753
Epoch 9/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4802 - accuracy: 0.7679
Epoch 10/10
24/24 [==============================] - 0s 3ms/step - loss: 0.4968 - accuracy: 0.7487
[0.4967737495899200

In [29]:
# Method 2 : Over Sampling

In [30]:
count_class_0, count_class_1

(5163, 1869)

In [31]:
df_class_1_overSampled = df_class_1.sample(count_class_0, replace = True)

In [32]:
df_overSampled = pd.concat([df_class_0, df_class_1_overSampled], axis=0)

In [33]:
print("OverSampling : ")
print(df_overSampled.Churn.value_counts())

OverSampling : 
0    5163
1    5163
Name: Churn, dtype: int64


In [34]:
x2 = df_overSampled.drop('Churn', axis=1)
y2 = df_overSampled['Churn']

In [35]:
x_train, x_test, y_train, y_test = train_test_split(x2, y2, test_size=0.2, random_state=13, stratify=y2)

In [36]:
y_train.value_counts()

1    4130
0    4130
Name: Churn, dtype: int64

In [37]:
y_pred = ANN(x_train, y_train, x_test, y_test,'binary_crossentropy',-1)

Epoch 1/10
259/259 [==============================] - 2s 3ms/step - loss: 0.5273 - accuracy: 0.7466
Epoch 2/10
259/259 [==============================] - 1s 3ms/step - loss: 0.4940 - accuracy: 0.7608
Epoch 3/10
259/259 [==============================] - 1s 3ms/step - loss: 0.4878 - accuracy: 0.7616
Epoch 4/10
259/259 [==============================] - 1s 3ms/step - loss: 0.4830 - accuracy: 0.7648
Epoch 5/10
259/259 [==============================] - 1s 3ms/step - loss: 0.4794 - accuracy: 0.7672
Epoch 6/10
259/259 [==============================] - 1s 3ms/step - loss: 0.4758 - accuracy: 0.7685
Epoch 7/10
259/259 [==============================] - 1s 3ms/step - loss: 0.4725 - accuracy: 0.7719
Epoch 8/10
259/259 [==============================] - 1s 3ms/step - loss: 0.4706 - accuracy: 0.7757
Epoch 9/10
259/259 [==============================] - 1s 4ms/step - loss: 0.4657 - accuracy: 0.7776
Epoch 10/10
65/65 [==============================] - 0s 3ms/step - loss: 0.4620 - accuracy: 0.7759
[

In [38]:
#  SMOTE

In [39]:
x = df_with_dummies.drop('Churn', axis = 1)
y = df_with_dummies['Churn']
y.value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [40]:
from imblearn.over_sampling import SMOTE

In [41]:
smote = SMOTE(sampling_strategy='minority')
x_sm, y_sm = smote.fit_resample(x,y)
y_sm.value_counts()

0    5163
1    5163
Name: Churn, dtype: int64

In [42]:
x_train, x_test, y_train, y_test = train_test_split(x_sm, y_sm, test_size=0.2, random_state=13, stratify=y_sm)

In [43]:
y_train.value_counts()

1    4130
0    4130
Name: Churn, dtype: int64

In [44]:
y_pred = ANN(x_train, y_train, x_test, y_test,'binary_crossentropy',-1)

Epoch 1/10
259/259 [==============================] - 2s 3ms/step - loss: 0.5368 - accuracy: 0.7473
Epoch 2/10
259/259 [==============================] - 1s 3ms/step - loss: 0.4793 - accuracy: 0.7705
Epoch 3/10
259/259 [==============================] - 1s 2ms/step - loss: 0.4722 - accuracy: 0.7736
Epoch 4/10
259/259 [==============================] - 1s 4ms/step - loss: 0.4659 - accuracy: 0.7762
Epoch 5/10
259/259 [==============================] - 1s 4ms/step - loss: 0.4609 - accuracy: 0.7794
Epoch 6/10
259/259 [==============================] - 1s 3ms/step - loss: 0.4551 - accuracy: 0.7800
Epoch 7/10
259/259 [==============================] - 1s 3ms/step - loss: 0.4503 - accuracy: 0.7841
Epoch 8/10
259/259 [==============================] - 1s 3ms/step - loss: 0.4460 - accuracy: 0.7862
Epoch 9/10
259/259 [==============================] - 1s 3ms/step - loss: 0.4419 - accuracy: 0.7925
Epoch 10/10
65/65 [==============================] - 0s 3ms/step - loss: 0.4226 - accuracy: 0.8049
[

In [45]:
# Method 4 : Use of Ensemble with undersampling

In [46]:
df_with_dummies.Churn.value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [47]:
x_ens = df_with_dummies.drop('Churn', axis = 1)
y_ens = df_with_dummies['Churn']

In [48]:
x_train, x_test, y_train, y_test = train_test_split(x_ens, y_ens, test_size=0.2, random_state=13, stratify=y_ens)

In [49]:
y_train.value_counts()

0    4130
1    1495
Name: Churn, dtype: int64

In [50]:
y_test.value_counts()

0    1033
1     374
Name: Churn, dtype: int64

In [51]:
4130/1495

2.762541806020067

In [52]:
1033/374

2.7620320855614975

In [53]:
df_train = x_train.copy()
df_train['Churn'] = y_train

In [54]:
df_train_class_0 = df_train[df_train.Churn == 0]
df_train_class_1 = df_train[df_train.Churn == 1]

In [55]:
df_train_class_0.shape, df_train_class_1.shape

((4130, 27), (1495, 27))

In [56]:
# df_train_underSampleForEnsebling_batch1 = df_train_class_0[:1109]

In [57]:
# df_train_batch1 = pd.concat([df_train_underSampleForEnsebling_batch1, df_train_class_1], axis = 0)

In [58]:
# df_train_underSampleForEnsebling_batch2 = df_train_class_0[1109: 2218]
# df_train_batch1 = pd.concat([df_train_underSampleForEnsebling_batch2, df_train_class_1], axis = 0)

In [59]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis = 0)
    
    x_train = df_train.drop('Churn', axis = 1)
    y_train = df_train.Churn
    
    return x_train, y_train

In [60]:
x_train, y_train = get_train_batch(df_train_class_0, df_train_class_1, 0, 1495)

In [61]:
x_train.shape

(2990, 26)

In [62]:
y_pred1 = ANN(x_train, y_train, x_test, y_test,'binary_crossentropy',-1)

Epoch 1/10
94/94 [==============================] - 1s 3ms/step - loss: 0.6159 - accuracy: 0.6538
Epoch 2/10
94/94 [==============================] - 0s 3ms/step - loss: 0.5217 - accuracy: 0.7391
Epoch 3/10
94/94 [==============================] - 0s 3ms/step - loss: 0.5086 - accuracy: 0.7475
Epoch 4/10
94/94 [==============================] - 0s 3ms/step - loss: 0.5013 - accuracy: 0.7472
Epoch 5/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4977 - accuracy: 0.7542
Epoch 6/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4925 - accuracy: 0.7559
Epoch 7/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4908 - accuracy: 0.7525
Epoch 8/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4877 - accuracy: 0.7552
Epoch 9/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4871 - accuracy: 0.7535
Epoch 10/10
44/44 [==============================] - 1s 3ms/step - loss: 0.4642 - accuracy: 0.7690
[0.4642467200756073

In [63]:
x_train, y_train = get_train_batch(df_train_class_0, df_train_class_1, 1495, 2990)
y_pred2 = ANN(x_train, y_train, x_test, y_test,'binary_crossentropy',-1)

Epoch 1/10
94/94 [==============================] - 1s 3ms/step - loss: 0.6375 - accuracy: 0.6488
Epoch 2/10
94/94 [==============================] - 0s 3ms/step - loss: 0.5363 - accuracy: 0.7452
Epoch 3/10
94/94 [==============================] - 0s 3ms/step - loss: 0.5086 - accuracy: 0.7548
Epoch 4/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4988 - accuracy: 0.7518
Epoch 5/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4942 - accuracy: 0.7589
Epoch 6/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4919 - accuracy: 0.7528
Epoch 7/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4887 - accuracy: 0.7585
Epoch 8/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4866 - accuracy: 0.7565
Epoch 9/10
94/94 [==============================] - 0s 3ms/step - loss: 0.4853 - accuracy: 0.7639
Epoch 10/10
44/44 [==============================] - 0s 3ms/step - loss: 0.4407 - accuracy: 0.7882
[0.4406510591506958

In [64]:
x_train, y_train = get_train_batch(df_train_class_0, df_train_class_1, 2990, 4130)
y_pred3 = ANN(x_train, y_train, x_test, y_test,'binary_crossentropy',-1)

Epoch 1/10
83/83 [==============================] - 1s 2ms/step - loss: 0.6465 - accuracy: 0.6167
Epoch 2/10
83/83 [==============================] - 0s 2ms/step - loss: 0.5369 - accuracy: 0.7507
Epoch 3/10
83/83 [==============================] - 0s 3ms/step - loss: 0.4938 - accuracy: 0.7617
Epoch 4/10
83/83 [==============================] - 0s 4ms/step - loss: 0.4821 - accuracy: 0.7639
Epoch 5/10
83/83 [==============================] - 0s 3ms/step - loss: 0.4755 - accuracy: 0.7677
Epoch 6/10
83/83 [==============================] - 0s 3ms/step - loss: 0.4708 - accuracy: 0.7738
Epoch 7/10
83/83 [==============================] - 0s 3ms/step - loss: 0.4675 - accuracy: 0.7700
Epoch 8/10
83/83 [==============================] - 0s 4ms/step - loss: 0.4645 - accuracy: 0.7738
Epoch 9/10
83/83 [==============================] - 0s 3ms/step - loss: 0.4611 - accuracy: 0.7799
Epoch 10/10
44/44 [==============================] - 0s 3ms/step - loss: 0.5194 - accuracy: 0.7349
[0.519393801689148,

In [65]:
# Majority Vote
vote1 = 0
vote2 = 0
vote3 = 1

vote1+vote2+vote3

1

In [66]:
len(y_pred3)

1407

In [67]:
y_pred_final = y_pred1.copy()

for i in range(len(y_pred1)):
    n_ones = y_pred1[i]+y_pred2[i]+y_pred3[i]
    
    if n_ones>1 :
        y_pred_final[i]=1
    else :
        y_pred_final[i]=0

In [68]:
print(classification_report(y_test, y_pred_final))

              precision    recall  f1-score   support

           0       0.91      0.76      0.83      1033
           1       0.54      0.80      0.65       374

    accuracy                           0.77      1407
   macro avg       0.73      0.78      0.74      1407
weighted avg       0.82      0.77      0.78      1407



In [69]:
# Plotting Confusion Matrix

# import seaborn as sn
# cm = tf.math.confusion_matrix(labels=y_test, predictions=y_predicted_final)

# plt.figure(figsize=(10,7))
# sn.heatmap(cm, annot=True, fmt='d')
# plt.xlabel('Predicted')
# plt.ylabel('Truth')